<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>PySpark e Apache Kafka Para Processamento de Dados em Batch e Streaming</font>
## <font color='blue'>Projeto 3</font>
### <font color='blue'>Pipeline de Limpeza e Transformação Para Aplicações de IA com PySpark SQL</font>

## Pacotes Python Usados no Projeto

In [1]:
# Imports
import os
import pyspark
import pandas as pd
import numpy as np
import pyspark.sql.functions as F
import pyspark.ml.evaluation as evals
import pyspark.ml.tuning as tune
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import  VectorAssembler
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import round

In [2]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Criando a Sessão Spark e Definindo o Nível de Log

In [3]:
# Cria a sessão
spark = SparkSession.builder.appName('Projeto3-Exp').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/29 22:32:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/29 22:32:55 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
# Cria a sessão e especifica parâmetros do cluster
spark = SparkSession.builder \
    .appName('Projeto3-Exp') \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.cores', '2') \
    .getOrCreate()

24/11/29 22:33:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
# Cria a sessão Spark com YARN como gerenciador de recursos e especifica parâmetros do cluster
spark = SparkSession.builder \
    .appName('Projeto3-Exp') \
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.memory', '1g') \
    .config('spark.executor.cores', '2') \
    .getOrCreate()

24/10/20 16:49:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


**SparkSession.builder \:** Inicia a construção de uma nova sessão Spark. A SparkSession é a entrada principal para a programação com Spark SQL e fornece uma interface unificada para configurar a aplicação.

**.appName('Projeto3-Exp') \:** Define o nome da aplicação, que será exibido na interface do YARN e do Spark UI. Isso ajuda a identificar a aplicação no cluster.

**.master('yarn') \:** Define o gerenciador de recursos do cluster. Ao especificar 'yarn', você informa ao Spark que deve usar o YARN para gerenciar os recursos da aplicação (como memória e núcleos de CPU).

**.config('spark.submit.deployMode', 'client') \:** Define o modo de execução do driver:

- 'client': O driver é executado na máquina local onde o código foi iniciado. Isso é comum em testes ou quando você precisa de interação direta com o driver.

- 'cluster': O driver é executado em um dos nós do cluster gerenciado pelo YARN, sendo mais adequado para produção, pois mantém a aplicação independente da máquina de origem. ESSE MODO NÃO FUNCIONA VIA JUPYTER NOTEBOOK.

**.config('spark.driver.memory', '4g') \:** Define a quantidade de memória alocada para o processo do driver (4 GB neste caso). O driver é responsável por gerenciar a execução da aplicação.

**.config('spark.executor.memory', '1g') \:** Define a quantidade de memória alocada para cada executor (1 GB neste caso). Os executores são os processos que executam as tarefas distribuídas pelo cluster.

**.config('spark.executor.cores', '2') \:** Define o número de núcleos de CPU alocados para cada executor (2 núcleos por executor). Isso influencia o paralelismo do processamento das tarefas.

**.getOrCreate()**: Finaliza a construção da sessão Spark e retorna a sessão criada. Se uma sessão com as mesmas configurações já estiver ativa, ela será reutilizada.

In [5]:
# Define o nível de log
spark.sparkContext.setLogLevel("ERROR")

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
## Carregando os Datasets a Partir do HDFS

In [6]:
# Carrega o arquivo 1
df_dsa_aeroportos = spark.read.csv("/opt/spark/data/dataset1.csv", header = True)

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 1]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [8]:
type(df_dsa_aeroportos)

pyspark.sql.dataframe.DataFrame

In [16]:
df_dsa_aeroportos.show(10, truncate=False)

+---+------------------------------+----------+------------+----+---+---+
|faa|name                          |lat       |lon         |alt |tz |dst|
+---+------------------------------+----------+------------+----+---+---+
|04G|Lansdowne Airport             |41.1304722|-80.6195833 |1044|-5 |A  |
|06A|Moton Field Municipal Airport |32.4605722|-85.6800278 |264 |-5 |A  |
|06C|Schaumburg Regional           |41.9893408|-88.1012428 |801 |-6 |A  |
|06N|Randall Airport               |41.431912 |-74.3915611 |523 |-5 |A  |
|09J|Jekyll Island Airport         |31.0744722|-81.4277778 |11  |-4 |A  |
|0A9|Elizabethton Municipal Airport|36.3712222|-82.1734167 |1593|-4 |A  |
|0G6|Williams County Airport       |41.4673056|-84.5067778 |730 |-5 |A  |
|0G7|Finger Lakes Regional Airport |42.8835647|-76.7812318 |492 |-5 |A  |
|0P2|Shoestring Aviation Airfield  |39.7948244|-76.6471914 |1000|-5 |U  |
|0S9|Jefferson County Intl         |48.0538086|-122.8106436|108 |-8 |A  |
+---+------------------------------+--

In [10]:
# Carrega o arquivo 2
df_dsa_voos = spark.read.csv("/opt/spark/data/dataset2.csv", header = True)

In [11]:
df_dsa_voos.show(10)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [12]:
# Carrega o arquivo 3
df_dsa_aeronaves = spark.read.csv("/opt/spark/data/dataset3.csv", header = True)

In [17]:
df_dsa_aeronaves.show(20, truncate=False)

+-------+----+-----------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|type                   |manufacturer    |model   |engines|seats|speed|engine   |
+-------+----+-----------------------+----------------+--------+-------+-----+-----+---------+
|N102UW |1998|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N103US |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N104UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N105UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N107US |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N108UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N109UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N110UW |1999|Fixed wing multi engine|AIRBUS INDUS

Vamos converter esses dados para o formato:

- Dados de entrada --> ['month', 'air_time', 'carr_fact', 'dest_fact', 'plane_age'] como o vetor features.
- Dados de saída --> ['is_late'] com o nome label.

E então usaremos os dados nesse formato para treinar e avaliar dois modelos de Machine Learning. Escolheremos o melhor modelo e então criaremos o job de automação do processo de treinamento no cluster Spark.

## Continuaremos no Próximo Capítulo.

In [ ]:
%reload_ext watermark
%watermark -a "Data Science Academy"

# Fim